In [1]:
!pip install tensorflow-recommenders
!pip install tensorflow
!pip install pandas
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder




In [3]:
# Load the dataset (adjust the path based on your environment)
df = pd.read_csv('workout_dataset.csv')

# Display the first few rows of the dataset
print(df.head())

# Check for missing values
print(df.isnull().sum())


   id  age  height  weight  fitness_goal workout_level  \
0   0   23     169      87  build muscle      advanced   
1   1   50     182      87   lose weight      beginner   
2   2   33     177      62   lose weight  intermediate   
3   3   25     172      75  build muscle      beginner   
4   4   29     163      51      maintain      beginner   

                       health_conditions  \
0                          Heart Disease   
1                            Sleep Apnea   
2  Gastroesophageal Reflux Disease(GERD)   
3                                 Anemia   
4                            Cholesterol   

                          recommend_workout  \
0                   crunches, lunges, plank   
1  lunges, plank, pushups, crunches, squats   
2  lunges, plank, pushups, crunches, squats   
3            squats, plank, lunges, pushups   
4                   crunches, lunges, plank   

                                         repetitions  \
0               crunches: 35, lunges: 20, plank

In [4]:
# Fill any missing values in the 'health_conditions' column with 'unknown'
df['health_conditions'] = df['health_conditions'].fillna('unknown')

# Drop unnecessary columns if any (e.g., 'id')
df = df.drop(columns=['id'], errors='ignore')  # Assuming 'id' column exists, modify if needed


In [5]:
# Check for missing values
print(df.isnull().sum())

age                  0
height               0
weight               0
fitness_goal         0
workout_level        0
health_conditions    0
recommend_workout    0
repetitions          0
sets                 0
image_url            0
dtype: int64


In [6]:


# Load the modified dataset (the one you created with repetitions, sets, and image URLs)
df = pd.read_csv('workout_dataset.csv')

# Define features and target variable
# We only consider numeric and categorical variables as input features, excluding repetition counts, sets, and images
X = df[['age', 'height', 'weight']]  # Numerical features
y = df['recommend_workout']  # Target variable

# One-hot encode the categorical variables (fitness_goal, workout_level, health_conditions)
encoder = OneHotEncoder(sparse_output=False)

# Assuming fitness_goal, workout_level, and health_conditions are in the dataset
encoded_categorical = encoder.fit_transform(df[['fitness_goal', 'workout_level', 'health_conditions']])

# Combine numerical and one-hot encoded categorical features
X = np.hstack((X.values, encoded_categorical))

# Encode target variable (recommend_workout) into numeric values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert the encoded target variable (recommend_workout) to a categorical format
num_classes = len(np.unique(y_encoded))
y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes)

# Now you can use X and y_categorical to train a neural network model.
print(f"Shape of input features X: {X.shape}")
print(f"Shape of target variable y (categorical): {y_categorical.shape}")


Shape of input features X: (70000, 18)
Shape of target variable y (categorical): (70000, 5)


In [7]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),  # Input layer
    tf.keras.layers.Dense(128, activation='relu'),  # First hidden layer with 128 units
    tf.keras.layers.Dense(64, activation='relu'),  # Second hidden layer with 64 units
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Output layer with softmax for classification
])



In [8]:
# Compile the model with Adam optimizer and categorical cross-entropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for 10 epochs with training data and validate on test data
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Epoch 1/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7550 - loss: 0.8763 - val_accuracy: 0.9621 - val_loss: 0.1110
Epoch 2/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9680 - loss: 0.0875 - val_accuracy: 0.9611 - val_loss: 0.1147
Epoch 3/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9728 - loss: 0.0716 - val_accuracy: 0.9689 - val_loss: 0.0811
Epoch 4/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9759 - loss: 0.0589 - val_accuracy: 0.9796 - val_loss: 0.0474
Epoch 5/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9783 - loss: 0.0514 - val_accuracy: 0.9757 - val_loss: 0.0512
Epoch 6/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9808 - loss: 0.0463 - val_accuracy: 0.9829 - val_loss: 0.0394
Epoch 7/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9814 - loss: 0.0430 - val_accuracy: 0.9796 - val_loss: 0.0531
Epoch 8/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9823 - loss: 0.0405 -

In [9]:
import re
from IPython.display import display, HTML

# Define a dictionary mapping each exercise to its respective image URL
exercise_image_mapping = {
    'lunges': 'https://images.healthshots.com/healthshots/en/uploads/2024/05/02174153/Lunges-770x436.jpg',
    'plank': 'https://thumbs.dreamstime.com/b/fitness-training-athletic-sporty-man-doing-plank-exercise-gym-yoga-class-exercising-workout-70718580.jpg',
    'pushups': 'https://lasvegastoppicks.com/wp-content/uploads/2023/07/Pushups-on-Las-Vegas-Top-Picks.jpg',
    'crunches': 'https://fitnessdrum.com/wp-content/uploads/2021/09/oblique-crunches-min.png',
    'squats': 'https://media1.popsugar-assets.com/files/thumbor/uymLdLwcsZuQzm4oFGNFY2QLl5g=/fit-in/792x792/filters:format_auto():upscale()/2023/12/21/819/n/1922729/tmp_UyrHy8_feb4a42e2a9b21aa_PS23_Fitness_Workout_13_Move_06_Goblet_Squat.jpg'
}

# Function to parse sets and repetitions for a specific workout
def get_sets_reps_for_workout(sets_col, reps_col, workout_name):
    # Regex pattern to find the sets and reps for the specific workout
    sets_pattern = rf"{workout_name}: (\d+)"
    reps_pattern = rf"{workout_name}: (\d+\w*)"

    # Find sets for the workout
    sets_match = re.search(sets_pattern, sets_col)
    sets = sets_match.group(1) if sets_match else "N/A"

    # Find repetitions for the workout
    reps_match = re.search(reps_pattern, reps_col)
    reps = reps_match.group(1) if reps_match else "N/A"

    return sets, reps

# Function to display each recommended workout along with its sets, repetitions, and image
def display_recommended_workouts(row):
    # Split the recommended workouts
    workouts = row['recommend_workout'].split(', ')

    html_content = "<div style='font-family: Arial, sans-serif;'>"
    html_content += "<h3>The Recommended Workouts are:</h3>"

    for workout in workouts:
        # Fetch sets and repetitions for this workout
        sets, reps = get_sets_reps_for_workout(row['sets'], row['repetitions'], workout)

        # Get the image URL for the workout
        image_url = exercise_image_mapping.get(workout.lower(), "https://via.placeholder.com/150")

        # Build HTML content for each workout
        html_content += f"""
        <div style="display: flex; align-items: center; margin-bottom: 20px;">
            <div style="flex: 1;">
                <p><strong>Workout:</strong> {workout.capitalize()}</p>
                <p><strong>Sets:</strong> {sets}</p>
                <p><strong>Repetitions:</strong> {reps}</p>
            </div>
            <div style="flex: 1; text-align: right;">
                <img src="{image_url}" alt="{workout}" style="width: 150px; height: auto;">
            </div>
        </div>
        <hr style="border: 1px solid #ddd;">
        """

    html_content += "</div>"

    # Display the HTML content
    display(HTML(html_content))

# Example usage for a single row (replace 'df' with your DataFrame name)
row = df.iloc[0]  # Adjust the index as needed to select a specific row
display_recommended_workouts(row)


In [10]:
import re
from IPython.display import display, HTML

# Define a dictionary mapping each exercise to its respective image URL
exercise_image_mapping = {
    'lunges': 'https://images.healthshots.com/healthshots/en/uploads/2024/05/02174153/Lunges-770x436.jpg',
    'plank': 'https://thumbs.dreamstime.com/b/fitness-training-athletic-sporty-man-doing-plank-exercise-gym-yoga-class-exercising-workout-70718580.jpg',
    'pushups': 'https://lasvegastoppicks.com/wp-content/uploads/2023/07/Pushups-on-Las-Vegas-Top-Picks.jpg',
    'crunches': 'https://fitnessdrum.com/wp-content/uploads/2021/09/oblique-crunches-min.png',
    'squats': 'https://media1.popsugar-assets.com/files/thumbor/uymLdLwcsZuQzm4oFGNFY2QLl5g=/fit-in/792x792/filters:format_auto():upscale()/2023/12/21/819/n/1922729/tmp_UyrHy8_feb4a42e2a9b21aa_PS23_Fitness_Workout_13_Move_06_Goblet_Squat.jpg'
}

# Function to parse sets and repetitions for a specific workout
def get_sets_reps_for_workout(sets_col, reps_col, workout_name):
    # Regex pattern to find the sets and reps for the specific workout
    sets_pattern = rf"{workout_name}: (\d+)"
    reps_pattern = rf"{workout_name}: (\d+\w*)"

    # Find sets for the workout
    sets_match = re.search(sets_pattern, sets_col)
    sets = sets_match.group(1) if sets_match else "N/A"

    # Find repetitions for the workout
    reps_match = re.search(reps_pattern, reps_col)
    reps = reps_match.group(1) if reps_match else "N/A"

    return sets, reps

# Function to display each recommended workout along with its sets, repetitions, and image
def display_recommended_workouts(row):
    # Split the recommended workouts
    workouts = row['recommend_workout'].split(', ')

    html_content = "<div style='font-family: Arial, sans-serif;'>"
    html_content += "<h3>The Recommended Workouts are:</h3>"

    for workout in workouts:
        # Fetch sets and repetitions for this workout
        sets, reps = get_sets_reps_for_workout(row['sets'], row['repetitions'], workout)

        # Get the image URL for the workout
        image_url = exercise_image_mapping.get(workout.lower(), "https://via.placeholder.com/150")

        # Build HTML content for each workout
        html_content += f"""
        <div style="display: flex; align-items: center; margin-bottom: 20px;">
            <div style="flex: 1;">
                <p><strong>Workout:</strong> {workout.capitalize()}</p>
                <p><strong>Sets:</strong> {sets}</p>
                <p><strong>Repetitions:</strong> {reps}</p>
            </div>
            <div style="flex: 1; text-align: right;">
                <img src="{image_url}" alt="{workout}" style="width: 200px; height: auto; border-radius: 8px; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);">
            </div>
        </div>
        <hr style="border: 1px solid #ddd;">
        """

    html_content += "</div>"

    # Display the HTML content
    display(HTML(html_content))

# Example usage for a single row (replace 'df' with your DataFrame name)
row = df.iloc[0]  # Adjust the index as needed to select a specific row
display_recommended_workouts(row)


In [14]:
import re
from IPython.display import display, HTML

# Define a dictionary mapping each exercise to its respective image URL
exercise_image_mapping = {
    'lunges': 'https://images.healthshots.com/healthshots/en/uploads/2024/05/02174153/Lunges-770x436.jpg',
    'plank': 'https://thumbs.dreamstime.com/b/fitness-training-athletic-sporty-man-doing-plank-exercise-gym-yoga-class-exercising-workout-70718580.jpg',
    'pushups': 'https://lasvegastoppicks.com/wp-content/uploads/2023/07/Pushups-on-Las-Vegas-Top-Picks.jpg',
    'crunches': 'https://fitnessdrum.com/wp-content/uploads/2021/09/oblique-crunches-min.png',
    'squats': 'https://media1.popsugar-assets.com/files/thumbor/uymLdLwcsZuQzm4oFGNFY2QLl5g=/fit-in/792x792/filters:format_auto():upscale()/2023/12/21/819/n/1922729/tmp_UyrHy8_feb4a42e2a9b21aa_PS23_Fitness_Workout_13_Move_06_Goblet_Squat.jpg'
}

# Function to parse sets and repetitions for a specific workout
def get_sets_reps_for_workout(sets_col, reps_col, workout_name):
    # Regex pattern to find the sets and reps for the specific workout
    sets_pattern = rf"{workout_name}: (\d+)"
    reps_pattern = rf"{workout_name}: (\d+\w*)"

    # Find sets for the workout
    sets_match = re.search(sets_pattern, sets_col)
    sets = sets_match.group(1) if sets_match else "N/A"

    # Find repetitions for the workout
    reps_match = re.search(reps_pattern, reps_col)
    reps = reps_match.group(1) if reps_match else "N/A"

    return sets, reps

# Function to display each recommended workout along with its sets, repetitions, and image
def display_recommended_workouts(row):
    # Split the recommended workouts
    workouts = row['recommend_workout'].split(', ')

    html_content = "<div style='font-family: Arial, sans-serif;'>"
    html_content += "<h3>The Recommended Workouts are:</h3>"

    for workout in workouts:
        # Fetch sets and repetitions for this workout
        sets, reps = get_sets_reps_for_workout(row['sets'], row['repetitions'], workout)

        # Get the image URL for the workout
        image_url = exercise_image_mapping.get(workout.lower(), "https://via.placeholder.com/150")

        # Build HTML content for each workout
        html_content += f"""
        <div style="display: flex; align-items: center; margin-bottom: 15px;">
            <div style="flex: 1;">
                <p><strong>Workout:</strong> {workout.capitalize()}</p>
                <p><strong>Sets:</strong> {sets}</p>
                <p><strong>Repetitions:</strong> {reps}</p>
            </div>
            <div style="flex: 0 0 200px; text-align: right; padding-left: 3px;">
                <img src="{image_url}" alt="{workout}" style="width: 200px; height: auto; border-radius: 8px;">
            </div>
        </div>
        <hr style="border: 1px solid #ddd;">
        """

    html_content += "</div>"

    # Display the HTML content
    display(HTML(html_content))

# Example usage for a single row (replace 'df' with your DataFrame name)
row = df.iloc[0]  # Adjust the index as needed to select a specific row
display_recommended_workouts(row)
